In [ ]:
import cobra

def tsv2model(metabolites_file_path, reactions_file_path, model_name = 'tsv_model'):
    import pandas as pd
    from cobra import Model, Reaction, Metabolite
    
    model = Model(model_name)
    
    # Create a dictionary that stores the metabolites information
    #excel_model = pd.read_excel(file_path, sheet_name="Metabolite List")
    model_metabolites = pd.read_csv(metabolites_file_path, sep='\t')
    metabolites = {}
    
    for i in range(len(model_metabolites)):
        metabolites[model_metabolites.iloc[i,0]] = Metabolite(model_metabolites.iloc[i,0], 
                                             formula=model_metabolites.iloc[i,3], 
                                             name=model_metabolites.iloc[i,1], 
                                             compartment=model_metabolites.iloc[i,5])
    
    model_reactions = pd.read_csv(reactions_file_path, sep='\t').fillna("") # any empty cell is read in as a NaN float object. fillna() replaces any NaN float object with an empty string
    
    biomass_reaction = []

    for i in range(len(model_reactions)):
        dict_reactants = {}
        dict_products = {}
    
        reaction = Reaction(model_reactions.iloc[i,0])
        reaction.name = model_reactions.iloc[i,1]
        reaction.lower_bound = float(model_reactions.iloc[i,8])
        reaction.upper_bound = float(model_reactions.iloc[i,9])
    
        reaction_formula = model_reactions.iloc[i,2]
        
        if reaction_formula.endswith("<=>" or "=>"):
            reactants = reaction_formula.strip(" <=>" or " =>")
            products = [""]
            if isinstance(reactants, list) == 0:
                reactants = [reactants]
        elif reaction_formula.startswith("<=>" or "=>"):
            reactants = [""]
            products = reaction_formula.strip("<=>" or "=>")
            if isinstance(products, list) == 0:
                products = [products]
        elif " <=>" in reaction_formula:
            reactants = reaction_formula.split("<=>")[0]
            products = reaction_formula.split("<=>")[1]
            if isinstance(reactants, list) == 0:
                reactants = [reactants]
            if isinstance(products, list) == 0:
                products = [products]
        elif " =>" in reaction_formula:
            reactants = reaction_formula.split("=>")[0]
            products = reaction_formula.split("=>")[1]
            if isinstance(reactants, list) == 0:
                reactants = [reactants]
            if isinstance(products, list) == 0:
                products = [products]
    
        if len(reactants) == 1:
            reactants = str(reactants[0])
        else:
            print("reaction formula incorrect")
    
        if len(products) == 1:
            products = str(products[0])
        else:
            print("reaction formula incorrect")
    
    
        if "+" in reactants:
            reactants = reactants.split("+")
            for j in range(len(reactants)):
                reactants[j] = reactants[j].strip()
                if " " in reactants[j]:
                    coeff_reactant_split = reactants[j].split(" ")
                    reactant_coeff = -float(coeff_reactant_split[0])
                    reactant_species = coeff_reactant_split[1]
                    dict_reactants[metabolites[reactant_species]] = reactant_coeff
                else:
                    reactant_coeff = float(-1)
                    reactant_species = reactants[j]
                    dict_reactants[metabolites[reactant_species]] = reactant_coeff
        elif len(reactants) > 1:
            reactants = reactants.strip()
            if " " in reactants:
                coeff_reactant_split = reactants.split(" ")
                reactant_coeff = -float(coeff_reactant_split[0])
                reactant_species = coeff_reactant_split[1]
                dict_reactants[metabolites[reactant_species]] = reactant_coeff
            else:
                reactant_coeff = float(-1)
                reactant_species = reactants
                dict_reactants[metabolites[reactant_species]] = reactant_coeff
            
        if "+" in products:
            products = products.split("+")
            for j in range(len(products)):
                products[j] = products[j].strip()
                if " " in products[j]:
                    coeff_product_split = products[j].split(" ")
                    product_coeff = float(coeff_product_split[0])
                    product_species = coeff_product_split[1]
                    dict_products[metabolites[product_species]] = product_coeff
                else:
                    product_coeff = float(1)
                    product_species = products[j]
                    dict_products[metabolites[product_species]] = product_coeff
        elif len(products) > 1:
            products = products.strip()
            if " " in products:
                coeff_product_split = products.split(" ")
                product_coeff = float(coeff_product_split[0])
                product_species = coeff_product_split[1]
                dict_products[metabolites[product_species]] = product_coeff
            else:
                product_coeff = float(1)
                product_species = products
                dict_products[metabolites[product_species]] = product_coeff
            
        reaction.add_metabolites({**dict_reactants, **dict_products})
        model.add_reactions([reaction])
        
        if str(model_reactions.iloc[i,3]) != "":
            reaction.gene_reaction_rule = str(model_reactions.iloc[i,3]) # str() is important because a balnk cell is read in as a float
    
        if model_reactions.iloc[i,10] == 1:
            biomass_reaction.append(model_reactions.iloc[i,0])
    
    # Print model metrics
    print('%i reaction' % len(model.reactions))
    print('%i metabolites' % len(model.metabolites))
    print('%i genes' % len(model.genes))
    
    if len(biomass_reaction) == 1:
        print("Objective function is:")
        print(biomass_reaction[0])
        model.objective = biomass_reaction[0]
    else:
        print("Cannot assign objective function. No or multiple reactions specified as objective functions")
    
    return(model)
        

In [ ]:
import cobra

def xls2model(excel_file_path, model_name = 'xls_model'):
    import pandas as pd
    from cobra import Model, Reaction, Metabolite
    
    model = Model(name)
    
    # Create a dictionary that stores the metabolites information
    excel_model = pd.read_excel(file_path, sheet_name="Metabolite List")
    metabolites = {}
    
    for i in range(len(excel_model)):
        metabolites[excel_model.iloc[i,0]] = Metabolite(excel_model.iloc[i,0], 
                                             formula=excel_model.iloc[i,3], 
                                             name=excel_model.iloc[i,1], 
                                             compartment=excel_model.iloc[i,5])
    
    excel_model = pd.read_excel(file_path, sheet_name="Reaction List").fillna("") # any empty cell is read in as a NaN float object. fillna() replaces any NaN float object with an empty string
    
    biomass_reaction = []

    for i in range(len(excel_model)):
        dict_reactants = {}
        dict_products = {}
    
        reaction = Reaction(excel_model.iloc[i,0])
        reaction.name = excel_model.iloc[i,1]
        reaction.lower_bound = float(excel_model.iloc[i,8])
        reaction.upper_bound = float(excel_model.iloc[i,9])
    
        reaction_formula = excel_model.iloc[i,2]
        
        if reaction_formula.endswith("<=>" or "=>"):
            reactants = reaction_formula.strip(" <=>" or " =>")
            products = [""]
            if isinstance(reactants, list) == 0:
                reactants = [reactants]
        elif reaction_formula.startswith("<=>" or "=>"):
            reactants = [""]
            products = reaction_formula.strip("<=>" or "=>")
            if isinstance(products, list) == 0:
                products = [products]
        elif " <=>" in reaction_formula:
            reactants = reaction_formula.split("<=>")[0]
            products = reaction_formula.split("<=>")[1]
            if isinstance(reactants, list) == 0:
                reactants = [reactants]
            if isinstance(products, list) == 0:
                products = [products]
        elif " =>" in reaction_formula:
            reactants = reaction_formula.split("=>")[0]
            products = reaction_formula.split("=>")[1]
            if isinstance(reactants, list) == 0:
                reactants = [reactants]
            if isinstance(products, list) == 0:
                products = [products]
    
        if len(reactants) == 1:
            reactants = str(reactants[0])
        else:
            print("reaction formula incorrect")
    
        if len(products) == 1:
            products = str(products[0])
        else:
            print("reaction formula incorrect")
    
    
        if "+" in reactants:
            reactants = reactants.split("+")
            for j in range(len(reactants)):
                reactants[j] = reactants[j].strip()
                if " " in reactants[j]:
                    coeff_reactant_split = reactants[j].split(" ")
                    reactant_coeff = -float(coeff_reactant_split[0])
                    reactant_species = coeff_reactant_split[1]
                    dict_reactants[metabolites[reactant_species]] = reactant_coeff
                else:
                    reactant_coeff = float(-1)
                    reactant_species = reactants[j]
                    dict_reactants[metabolites[reactant_species]] = reactant_coeff
        elif len(reactants) > 1:
            reactants = reactants.strip()
            if " " in reactants:
                coeff_reactant_split = reactants.split(" ")
                reactant_coeff = -float(coeff_reactant_split[0])
                reactant_species = coeff_reactant_split[1]
                dict_reactants[metabolites[reactant_species]] = reactant_coeff
            else:
                reactant_coeff = float(-1)
                reactant_species = reactants
                dict_reactants[metabolites[reactant_species]] = reactant_coeff
            
        if "+" in products:
            products = products.split("+")
            for j in range(len(products)):
                products[j] = products[j].strip()
                if " " in products[j]:
                    coeff_product_split = products[j].split(" ")
                    product_coeff = float(coeff_product_split[0])
                    product_species = coeff_product_split[1]
                    dict_products[metabolites[product_species]] = product_coeff
                else:
                    product_coeff = float(1)
                    product_species = products[j]
                    dict_products[metabolites[product_species]] = product_coeff
        elif len(products) > 1:
            products = products.strip()
            if " " in products:
                coeff_product_split = products.split(" ")
                product_coeff = float(coeff_product_split[0])
                product_species = coeff_product_split[1]
                dict_products[metabolites[product_species]] = product_coeff
            else:
                product_coeff = float(1)
                product_species = products
                dict_products[metabolites[product_species]] = product_coeff
            
        reaction.add_metabolites({**dict_reactants, **dict_products})
        model.add_reactions([reaction])
        
        if str(excel_model.iloc[i,3]) != "":
            reaction.gene_reaction_rule = str(excel_model.iloc[i,3]) # str() is important because a balnk cell is read in as a float
    
        if excel_model.iloc[i,10] == 1:
            biomass_reaction.append(excel_model.iloc[i,0])
    
    # Print model metrics
    print('%i reaction' % len(model.reactions))
    print('%i metabolites' % len(model.metabolites))
    print('%i genes' % len(model.genes))
    
    if len(biomass_reaction) == 1:
        print("Objective function is:")
        print(biomass_reaction[0])
        model.objective = biomass_reaction[0]
    else:
        print("Cannot assign objective function. No or multiple reactions specified as objective functions")
    
    return(model)
        

In [ ]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in cobra_model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in cobra_model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in cobra_model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))
